In [1]:
from selenium import webdriver
import os, random, time
import os.path
import glob
import pandas as pd
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
#import facebook login info
from loginFB import fb_login
import pandas as pd
import random

topull = pd.read_csv("austin_links_input.csv", header = 0)

company_link = topull.links.tolist()

company_names = topull.companies.tolist()

In [2]:
from selenium import webdriver
from selenium.webdriver.common.proxy import *

#myProxy = "host:8080"

#proxy = Proxy({
#    'proxyType': ProxyType.MANUAL,
#    'httpProxy': myProxy,
#    'ftpProxy': myProxy,
#    'sslProxy': myProxy,
#    'noProxy': '' # set this value as desired
#    })

#driver = webdriver.Firefox(proxy=proxy)

# for remote
#caps = webdriver.DesiredCapabilities.FIREFOX.copy()
#proxy.add_to_capabilities(caps)

#driver = webdriver.Remote(desired_capabilities=caps)
#open up a web browser for crawling/scraping





In [4]:
driver = webdriver.Firefox()

#link to company reviews for Austin, TX
searchlink = 'https://www.glassdoor.com/Reviews/austin-reviews-SRCH_IL.0,6_IM60.htm'

#go to webaddress 
#driver.get(searchlink)
#login to site
fb_login(driver)
driver.get(searchlink)

#link to company reviews for Austin, TX

#--------------------------
#problems:
    #NA's for sub ratings when they arent actually missing
    #add the company name to it!!!
#---------------------------
#pages = range(1,26) #i am scraping in batches so glassdoor doesn't notice and block me...
#you can change pages to something that searches the number of pages and then do len(pages)

#update companies before running again
#need last 13+ pages of vrbo
#'https://www.glassdoor.com/Reviews/Vrbo-Reviews-E42523_P{}.htm', 
#'Vrbo',



#company_link = ['https://www.glassdoor.com/Reviews/Khoros-Reviews-E482039_P{}.htm',
#               'https://www.glassdoor.com/Reviews/SailPoint-Technologies-Reviews-E449696_P{}.htm',
#               'https://www.glassdoor.com/Reviews/Volusion-Reviews-E236848_P{}.htm',
#               'https://www.glassdoor.com/Reviews/Invenio-Solutions-Reviews-E366063_P{}.htm', 
#                'https://www.glassdoor.com/Reviews/SolarWinds-Reviews-E100286_P{}.htm',
#               'https://www.glassdoor.com/Reviews/teleNetwork-Reviews-E28737_P{}.htm',
#               'https://www.glassdoor.com/Reviews/Rackspace-Reviews-E11783_P{}.htm',
#               'https://www.glassdoor.com/Reviews/Mood-Reviews-E372752_P{}.htm']
#company_names = ['Khoros', 'SailPoint', 'Volusion', 'Invenio Solutions', 'SolarWinds',
#                'TeleNetwork', 'Rackspace', 'Mood Media']



#if you want to store as all one file, initiate here and append outside the loop
#positives_full = []
#negatives_full = []
#advice_to_management_full = []
#review_date_full = []
#mains_full = []
#author_full = []
#star_ratings_full = []
#overall_rating_full = []
#helpful_full = []
#job_title_full = []
#date_info_full = []
#sub_review_raitings_full = []

for link in range(len(company_link)):
    positives = []
    negatives = []
    advice_to_management = []
    review_date = []
    mains = []
    author = []
    star_ratings = []
    overall_rating = []
    helpful = []
    job_title = []
    date_info = []
    sub_review_raitings = []
    #changing number of pages to less (before had it at 25ish but was taking too long)
    for i in range(1,random.randint(10,12)): #attempting to throw off bots by varying page number
        source = company_link[link].format(i) #go to page number of the numner in loop you are in
        driver.get(source)
        # wait 10 sec for the elements to become available
        driver.implicitly_wait(15)  
        txts = driver.find_elements_by_class_name('mt-md') #finds all the text fields
        
        #click all of the "show more"s so that we get the full review
        for txt in txts:
            try:
                txt.find_element_by_css_selector('span.link').click()
                time.sleep(random.uniform(0, 2))
            except:
                continue
        soup = BeautifulSoup(driver.page_source,'lxml')
        date_text = soup.find_all('time', {'class' : 'date subtle small'}) #get all the dates of the reveiws
        for d in range(len(date_text)):
            date_info.append(date_text[d]['datetime'])
        flag = soup.find('div', {'class' : 'featuredFlag'})#find if there is a featured review
        if flag:
            date_info = ['NA'] + date_info #if there is, then make the first date NA
        review_date.append(date_info)
        review_boxes = soup.find_all('div', {'class' : 'col-sm-11 pl-sm-lg mx-0'})
        time.sleep(random.uniform(0, .5))
        #loop through each review
        for (review_num,box) in enumerate(review_boxes):
        #'mt-md' is a class name that identifies boxes which hold
        # the text on individual reviews. There are 2 in each review,
        # the first is the pros, the second is the cons. 
            main_boxes = soup.find('p', {'class' : 'mainText mb-0'}) #states how long they worked there
            time.sleep(random.uniform(0, .5))
            categories = box.find_all('div', {'class' : 'mt-md'}) #pros and cons
            time.sleep(random.uniform(0, .5))
            author_boxes = soup.find('span', {'class': 'authorJobTitle'})
            time.sleep(random.uniform(0, .5))
            sub_rating_boxes = soup.find('i', {'class': 'subRatingsDrop'}).next_element #getting the block of html code that has the subratigns
            time.sleep(random.uniform(0, .5))
            sub_categories_box = sub_rating_boxes.find_all('div', {'class': 'minor'})
            sub_star_numbers_box = sub_rating_boxes.find_all('span', {'class': 'gdBars'})

            #time.sleep(random.uniform(0, .5))
            #job title
            job_title_text = box.find('span', {'class': 'summary'}).text

            #if len(categories) > 0 and len(categories) < 3:
            #    pros_box = categories[0]
            #    cons_box = categories[1]
            #    pros_text = pros_box.find('p').next_sibling.text
            #    cons_text = cons_box.find('p').next_sibling.text
            #else:
            #    pros_text = 'NA'
            #    cons_text = 'NA'
            pros_box = categories[0]
            cons_box = categories[1]
            pros_text = pros_box.find('p').next_sibling.text
            cons_text = cons_box.find('p').next_sibling.text

            time.sleep(random.uniform(0, .5))
            author_rating_box = soup.find('span', {'class': 'rating'})#overall rating
            author_rating = author_rating_box.next_element['title'] 

            if soup.find('div', {'class': 'helpfulReviews'}):
                helpful_count = soup.find('div', {'class': 'helpfulReviews'}).next_element[-2]
            else:
                helpful_count = 0

            #advice to management
            #if not missing then add to advice text, otherwise NA
            if len(categories) <= 2:
                advice_text = 'NA'
            else:
                advice_box = categories[2]
                advice_text = advice_box.find('p').next_sibling.text
                advice_text = advice_box.find('p').next_sibling.text
            time.sleep(random.uniform(0, .5))
            #sub reviews
            scores = {'Work/Life Balance' : 'NA', 'Culture & Values':'NA', 'Career Opportunities':'NA', 'Compensation and Benefits':'NA', 'Senior Management': 'NA'}
            for cat, stars in zip(sub_categories_box, sub_star_numbers_box):
                starnum = stars['title']
                catname = cat.text
                try:
                    scores[catname] = starnum
                except:
                    print('Category {} not found in scores'.format(catname))
                    continue
            
            #append reviews from current page
            main_text = main_boxes.text
            author_text = author_boxes.text
            positives.append(pros_text)
            negatives.append(cons_text)
            mains.append(main_text)
            author.append(author_text)
            overall_rating.append(author_rating)
            helpful.append(helpful_count)
            advice_to_management.append(advice_text)
            job_title.append(job_title_text)
            #print(job_title_text, scores)
            sub_review_raitings.append(scores)

        time.sleep(random.uniform(2,6)) #waiting to go to next page attempting to throw off bot detectors
    
    #save data to dataframe    
    d1 = {'pros': positives, 'cons': negatives, 'time_at_co': mains, 'author_info': author, 'overall_rating': overall_rating, 'job_title' : job_title, 'helpful_count': helpful, 'date': date_info, 'advice_to_management': advice_to_management}
    df1 = pd.DataFrame(d1)
    sub_reviews = pd.DataFrame.from_dict(sub_review_raitings)
    data = pd.concat([df1, sub_reviews], axis = 1)
    data['company'] = [company_names[link]] * len(data)
    data.to_csv('GlassdoorData/Austin/{}.csv'.format(company_names[link]))
    
    time.sleep(random.uniform(30,120)) #sleep before moving to new company
    
#d2 = {'pros': positives_full, 'cons': negatives_full, 'time_at_co': mains_full, 'author_info': author_full, 'overall_rating': overall_rating_full, 'job_title' : job_title_full, 'helpful_count': helpful_full, 'date': date_info_full, 'advice_to_management': advice_to_management_full}
#df2 = pd.DataFrame(d2)
#sub_reviews2 = pd.DataFrame.from_dict(sub_review_raitings_full)
#data2 = pd.concat([df2, sub_reviews2], axis = 1)
#data2['company'] = [company_names[link]] * len(data2)
#data2.to_csv('june12_scrape.csv')
    

dont run

In [41]:
import random
range(,random.randint(24,27))

range(1, 27)

In [42]:
random.randint(3,7)

6

In [5]:
d1 = {'pros': positives, 'cons': negatives, 'time_at_co': mains, 'author_info': author, 'overall_rating': overall_rating, 'job_title' : job_title, 'helpful_count': helpful, 'date': date_info, 'advice_to_management': advice_to_management}
df1 = pd.DataFrame(d1)
sub_reviews = pd.DataFrame.from_dict(sub_review_raitings)
data = pd.concat([df1, sub_reviews], axis = 1)
data

#for n in range(len(company_name)):
#    print(company_name[n]_
#data['company'] = 
#data.to_csv('accruent_page1thru25.csv')

,advice_to_management,author_info,cons,date,helpful_count,job_title,overall_rating,pros,time_at_co,Career Opportunities,Compensation and Benefits,Culture & Values,Senior Management,Work/Life Balance
0,NA,Current Employee - Staff Software Engineer,Sometimes is like drinking from a firehose bcs...,NA,9,Great tech + great culture,3.6,"Always jumping on the latest tech, so opportun...",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
1,NA,Current Employee - Staff Software Engineer,there are not any cons in my mind. I enjoyed m...,Fri Apr 05 2019 09:34:26 GMT-0500 (Central Day...,9,Great Place,3.6,"Great work place environment, good pay and ben...",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
2,NA,Current Employee - Staff Software Engineer,It's a miracle anything on the site/app works....,Fri May 24 2019 13:06:01 GMT-0500 (Central Day...,9,"Politics, Micromanagement and no focus on qual...",3.6,"Snacks, drinks, PTO, Friendly employees, flexi...",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
3,Increase the amount of inclusion for lower lev...,Current Employee - Staff Software Engineer,There is a stark difference in quality support...,Sun May 05 2019 21:53:59 GMT-0500 (Central Day...,9,Great company for the right people,3.6,They pay reasonably well and the culture is fa...,I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
4,Work to educate your employees as to the goals...,Current Employee - Staff Software Engineer,Senior Management in different departments and...,Tue May 14 2019 08:46:56 GMT-0500 (Central Day...,9,Dream Job turned Disappointment,3.6,"3-4 Weeks Vacation, 2 weeks PTO, Paid Maternit...",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
5,NA,Current Employee - Staff Software Engineer,"Coming from a start up, it has grown under exp...",Tue Apr 30 2019 09:22:04 GMT-0500 (Central Day...,9,Great Place to work,3.6,"Great team members, great culture. Hosts many ...",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
6,NA,Current Employee - Staff Software Engineer,Little Growth or development,Thu Apr 25 2019 06:25:11 GMT-0500 (Central Day...,9,Great Jon,3.6,"Good Benefits, good atmosphere",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
7,NA,Current Employee - Staff Software Engineer,Peak call time hours can be challenging to tak...,Thu Apr 25 2019 06:59:41 GMT-0500 (Central Day...,9,Good Pay/Benefits,3.6,Good benefits. Pay is very competitive for a c...,I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
8,NA,Current Employee - Staff Software Engineer,Innovation is not at their core. If you want t...,Wed Apr 24 2019 19:23:36 GMT-0500 (Central Day...,9,Unplanned Growth,3.6,The company offers excellent benefits that riv...,I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0
9,Stop putting on a front for the public and sta...,Current Employee - Staff Software Engineer,It is hard to put into words and you have to b...,Tue Apr 16 2019 11:16:30 GMT-0500 (Central Day...,9,It Could Be Better,3.6,"Generous PTO, Paid Holidays, Sick Days, Snacks",I have been working at Vrbo full-time for more...,5.0,4.0,5.0,5.0,5.0


In [10]:
company_name[link]

IndexError: list index out of range

In [34]:
for link in range(len(company_link)):
    print(link)
    print(company_names[link])

0
Vrbo
1
Main Street Hub
2
Invenio Solutions
3
SolarWinds
4
Indeed
5
HostGator


In [6]:
data['company'] = [company_names[link]] * len(data)

In [13]:
data

,advice_to_management,author_info,cons,date,helpful_count,job_title,overall_rating,pros,time_at_co,Career Opportunities,Compensation and Benefits,Culture & Values,Senior Management,Work/Life Balance,company
0,NA,Current Employee - Product Marketing,"Like several growing companies, the pace can b...",NA,9,Great Place for Career Growth,3.6,It's a really exciting time to be part of Accr...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
1,NA,Current Employee - Product Marketing,* you have to pay for parking,Thu May 23 2019 11:48:07 GMT-0500 (Central Day...,9,Accruent New Orleans- Great company to work for!,3.6,* great office culture in New Orleans- fun eve...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
2,NA,Current Employee - Product Marketing,Entry level positions aren't compensated well.,Thu Jun 06 2019 12:25:19 GMT-0500 (Central Day...,9,Good Company,3.6,Culture of the New Orleans office is great. Be...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
3,Develop more industry solutions and integratio...,Current Employee - Product Marketing,There's some real challenges that being an acq...,Mon May 13 2019 09:57:14 GMT-0500 (Central Day...,9,"Fast growing, challenging, lots of freedom, gr...",3.6,The growth this company is experiencing is imp...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
4,Don't try to reward people when they leave and...,Current Employee - Product Marketing,There are a core set of behaviors (values) are...,Fri May 24 2019 11:55:15 GMT-0500 (Central Day...,9,"Good place to learn, then leave",3.6,Come to Accurent for the experience; leave for...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
5,DO NOT ditch culture for efficiency. DO NOT be...,Current Employee - Product Marketing,"Parking is difficult, but it's downtown New Or...",Tue May 07 2019 08:12:38 GMT-0500 (Central Day...,9,Growing software company with a great NOLA focus,3.6,New Orleans is fast becoming a hub for technol...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
6,NA,Current Employee - Product Marketing,There is a lack of transparency.,Mon May 06 2019 13:19:51 GMT-0500 (Central Day...,9,Facilities Coordinator,3.6,Accruent offers free snacks and team building.,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
7,If you can be more transparent to your employe...,Current Employee - Product Marketing,Salary is not market value and management is n...,Sun Apr 14 2019 10:07:06 GMT-0500 (Central Day...,9,A good place for a stepping stone,3.6,Lots of experience to be gained due to the lar...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
8,NA,Current Employee - Product Marketing,With the rapid growth and acquisition of compa...,Thu Mar 28 2019 03:19:54 GMT-0500 (Central Day...,9,Great place to gain experience,3.6,The opportunities are available to gain as muc...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo
9,Thank you for your open communication and incl...,Current Employee - Product Marketing,1. Being part of a company that acquires other...,Thu Mar 07 2019 10:30:42 GMT-0600 (Central Sta...,9,You should read this before applying..,3.6,I have been with Accruent New Orleans office s...,I have been working at Accruent full-time for ...,5.0,NA,5.0,5.0,NA,Vrbo


In [32]:
company_link

['https://www.glassdoor.com/Reviews/Vrbo-Reviews-E42523_P{}.htm',
 'https://www.glassdoor.com/Reviews/Main-Street-Hub-Reviews-E504406_P{}.htm',
 'https://www.glassdoor.com/Reviews/Invenio-Solutions-Reviews-E366063_P{}.htm',
 'https://www.glassdoor.com/Reviews/SolarWinds-Reviews-E100286_P{}.htm',
 'https://www.glassdoor.com/Reviews/Indeed-Reviews-E100561_P{}.htm',
 'https://www.glassdoor.com/Reviews/HostGator-com-Reviews-E245439_P{}.htm']

In [33]:
company_names

['Vrbo',
 'Main Street Hub',
 'Invenio Solutions',
 'SolarWinds',
 'Indeed',
 'HostGator']

In [31]:
company_link = ['https://www.glassdoor.com/Reviews/Vrbo-Reviews-E42523_P{}.htm', 
                'https://www.glassdoor.com/Reviews/Main-Street-Hub-Reviews-E504406_P{}.htm',
               'https://www.glassdoor.com/Reviews/Invenio-Solutions-Reviews-E366063_P{}.htm',
               'https://www.glassdoor.com/Reviews/SolarWinds-Reviews-E100286_P{}.htm',
               'https://www.glassdoor.com/Reviews/Indeed-Reviews-E100561_P{}.htm',
               'https://www.glassdoor.com/Reviews/HostGator-com-Reviews-E245439_P{}.htm']
company_names = ['Vrbo', 'Main Street Hub', 'Invenio Solutions', 'SolarWinds', 'Indeed', 'HostGator']


In [30]:
random.uniform(1, 5)

2.5846264871461453

In [3]:
import time
import random
time.sleep(random.uniform(4,8))

In [9]:
sub_reviews = pd.DataFrame.from_dict(sub_review_raitings)
data = pd.concat([df1, sub_reviews], axis = 1)
data
data.to_csv('{}_page1thru25.csv'.format(company_names[link]))

In [6]:
random.uniform(60,120)

100.9336630718542